In [2]:
from MOT import MOT
import numpy as np
import os
from BfTableGenerator import RansacCollector
from BfTableGenerator import TDmapLoader
from tqdm import tqdm
import open3d as op3
# from open3d.web_visualizer import draw
from Utils import *
import pandas as pd
from scipy.sparse import csr_matrix
import json
from scipy.optimize import linear_sum_assignment
import pandas as pd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
RawTrajs_0 = pd.read_csv('../RawLidarData/McCarranEvans_Train/OutputFile/OutputTrajs/Trajctories_0.csv')
RawTrajs_1 = pd.read_csv('../RawLidarData/McCarranEvans_Train/OutputFile/OutputTrajs/Trajctories_1.csv')
TrajGroups_0 = RawTrajs_0.groupby('ObjectID')
TrajGroups_1 = RawTrajs_1.groupby('ObjectID')
TrajGroups_0 = [g[1] for g in TrajGroups_0]
TrajGroups_1 = [g[1] for g in TrajGroups_1]

In [43]:
X_cur,Y_cur,X_next,Y_next,X_speed_cur,Y_speed_cur,X_speed_next,Y_speed_next = [],[],[],[],[],[],[],[]


In [44]:
for ind in tqdm(range(len(TrajGroups_0))):
    temp_group = TrajGroups_0[ind]
    for i in range(1,len(temp_group)):
        if np.isnan(temp_group.iloc[i,2])|(np.isnan(temp_group.iloc[i-1,2])):
            continue
        X_next.append(temp_group.iloc[i].Coord_X_Mea)
        X_cur.append(temp_group.iloc[i-1].Coord_X_Mea)
        Y_next.append(temp_group.iloc[i].Coord_Y_Mea)
        Y_cur.append(temp_group.iloc[i-1].Coord_Y_Mea)
        X_speed_next.append(temp_group.iloc[i].Speed_X)
        X_speed_cur.append(temp_group.iloc[i-1].Speed_X)
        Y_speed_next.append(temp_group.iloc[i].Speed_Y)
        Y_speed_cur.append(temp_group.iloc[i-1].Speed_Y)
            
for ind in tqdm(range(len(TrajGroups_1))):
    temp_group = TrajGroups_1[ind]
    for i in range(1,len(temp_group)):
        if np.isnan(temp_group.iloc[i,2])|(np.isnan(temp_group.iloc[i-1,2])):
            continue
        X_next.append(temp_group.iloc[i].Coord_X_Mea)
        X_cur.append(temp_group.iloc[i-1].Coord_X_Mea)
        Y_next.append(temp_group.iloc[i].Coord_Y_Mea)
        Y_cur.append(temp_group.iloc[i-1].Coord_Y_Mea)
        X_speed_next.append(temp_group.iloc[i].Speed_X)
        X_speed_cur.append(temp_group.iloc[i-1].Speed_X)
        Y_speed_next.append(temp_group.iloc[i].Speed_Y)
        Y_speed_cur.append(temp_group.iloc[i-1].Speed_Y)

100%|████████████████████████████████████████████████████████████████████████████████| 530/530 [01:11<00:00,  7.36it/s]


In [45]:
result = pd.DataFrame({
    'X_cur':X_cur,
    'Y_cur':Y_cur,
    'X_speed_cur':X_speed_cur,
    'Y_speed_cur':Y_speed_cur,
    'X_next':X_next,
    'Y_next':Y_next,
    'X_speed_next':X_speed_next,
    'Y_speed_next':Y_speed_next,
})

In [51]:
result.to_csv('./PosteriorModel/Train.csv',index = False)

In [27]:
Label,Attribute = [],[]
for ind,traj in tqdm(enumerate(Trajs)):
    if Lens[ind] > 2:
        for i in range(1,len(traj)):
            labels = traj.iloc[i,[0,1,4,5]]
            attributes = traj.iloc[i-1,[0,1,4,5]]
            Label.append(labels)
            Attribute.append(attributes)
    else:
        continue 

2658it [00:55, 47.61it/s]


In [28]:
Label = pd.concat(Label,axis = 1).T
Attribute = pd.concat(Attribute,axis = 1).T

In [56]:
Label.to_csv('../PosteriorData/Label.csv',index = False)
Attribute.to_csv('../PosteriorData/Attribute.csv',index = False)

In [2]:
os.getcwd()

'e:\\Lidar-Tracker\\Lidar-Tracker'

In [4]:
params = {
    'd':1.2,
    'thred_s':0.3,
    'N':20,
    'delta_thred' : 1e-3,
    'step':0.1,
    'win_size':(5,11),
    'eps': 1.8,
    'min_samples':15,
    'missing_thred':7,
    'ending_frame' : 17950,
    'background_update_frame':2000,
    'save_pcd' : True,
    'save_Azimuth_Laser_info' : False,
    'result_type':'merged'
}

In [2]:
from MOT_NN import MOT_NN

tensor([[-11.8579, -24.2041]], grad_fn=<AddmmBackward>)


In [5]:
input_path = '../RawLidarData/Veteran'
dir_lis = os.listdir(input_path)
pcap_path = 'None'
for f in dir_lis:
    if 'pcap' in f.split('.'):
        pcap_path = os.path.join(input_path,f)
if pcap_path == 'None':
    print('Pcap file is not detected')
output_file_path = '../RawLidarData/Veteran'
config_path = os.path.join(input_path,'config.json')
ref_LLH_path,ref_xyz_path = os.path.join(input_path,'LLE_ref.csv'),os.path.join(input_path,'xyz_ref.csv')
ref_LLH,ref_xyz = np.array(pd.read_csv(ref_LLH_path)),np.array(pd.read_csv(ref_xyz_path))
ref_LLH[:,[0,1]] = ref_LLH[:,[0,1]] * np.pi/180
ref_LLH[:,2] = ref_LLH[:,2]/3.2808

# TrackingModel Path
model_path = './PosteriorModel/saved_model.pth' 

# with open(config_path) as f:
#     params = json.load(f)
mot = MOT_NN(pcap_path,output_file_path,model_path,**params)
mot.initialization()
mot.mot_tracking()
mot.save_result(ref_LLH,ref_xyz)

  0%|          | 2/2000 [00:00<01:40, 19.84it/s]

Frame update frame:2000 Initialization...


100%|██████████| 2000/2000 [01:30<00:00, 22.18it/s]


Generating Threshold Map


Tracking 2000 frame:  11%|█         | 1999/17949 [03:51<34:29,  7.71it/s]

...

Tracking 16000 frame:  89%|████████▉ | 15999/17949 [33:54<03:33,  9.13it/s]  

Generating Threshold Map


  2%|▏         | 61/2532 [00:00<00:04, 602.12it/s]

Generating Traj Files...


100%|██████████| 2532/2532 [00:04<00:00, 620.38it/s]
